<a href="https://colab.research.google.com/github/ErmakovAV/rec_sys/blob/main/hw_webinar_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import implicit

In [ ]:
!git clone https://github.com/ErmakovAV/rec_sys.git

fatal: destination path 'rec_sys' already exists and is not an empty directory.


In [ ]:
import metrics
dir(metrics)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'ap_k',
 'hit_rate',
 'hit_rate_at_k',
 'money_precision_at_k',
 'np',
 'precision',
 'precision_at_k',
 'recall',
 'recall_at_k',
 'reciprocal_rank']

In [ ]:
from metrics import precision_at_k

In [ ]:
precision_at_k

<function metrics.precision_at_k(recommended_list, bought_list, k=5)>

In [ ]:
import recommenders
dir(recommenders)

['AlternatingLeastSquares',
 'ItemItemRecommender',
 'MainRecommender',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bm25_weight',
 'csr_matrix',
 'np',
 'pd',
 'tfidf_weight']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from rec_sys.src.metrics import precision_at_k, recall_at_k
from rec_sys.src.utils import prefilter_items
from rec_sys.src.recommenders import MainRecommender

In [ ]:
data = pd.read_csv('/content/retail_train.csv')
item_features = pd.read_csv('/content/product.csv')
user_features = pd.read_csv('/content/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [ ]:
data_train_lvl_1["item_id"].count()

441033

In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()
n_items_before

42462

In [ ]:
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
data_train_lvl_1

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906.0,1.0,1.39,364.0,-0.60,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142.0,1.0,0.82,364.0,0.00,1631.0,1.0,0.0,0.0
2,2375,26984851472,1,1036325.0,1.0,0.99,364.0,-0.30,1631.0,1.0,0.0,0.0
4,2375,26984851472,1,8160430.0,1.0,1.50,364.0,-0.39,1631.0,1.0,0.0,0.0
5,2375,26984851516,1,826249.0,2.0,1.98,364.0,-0.60,1642.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
443333,1971,29300151035,173,7442008.0,1.0,3.19,310.0,0.00,1220.0,25.0,0.0,0.0
443335,1971,29300151035,173,9526411.0,1.0,2.14,310.0,0.00,1220.0,25.0,0.0,0.0
443337,414,29300152743,173,955043.0,1.0,0.59,310.0,0.00,1736.0,25.0,0.0,0.0
443339,998,29300153341,173,935317.0,1.0,7.68,310.0,0.00,1922.0,25.0,0.0,0.0


In [ ]:
data_train_lvl_1["item_id"].count()

231456

In [ ]:
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 42462 to 2614


In [ ]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2423 [00:00<?, ?it/s]

In [ ]:
recommender.get_als_recommendations(2375, N=200)[:8]

In [ ]:
recommender.get_own_recommendations(2375, N=200)[:8]

In [ ]:
recommender.get_similar_items_recommendation(2375, N=200)[:8]

In [ ]:
recommender.get_similar_users_recommendation(2375, N=200)[:8]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[854920.0, 856942.0, 912676.0, 920724.0, 93294..."
1,2,"[891440.0, 1032961.0, 1099397.0, 1127831.0, 64..."


In [ ]:
users_train = data_train_lvl_1['user_id'].tolist()
users_valid = result_lvl_1['user_id'].tolist()
new_users = list(set(users_valid) - set(users_train))
all_users = list(set(users_valid) & set(users_train))

result_lvl_1 = result_lvl_1[~result_lvl_1['user_id'].isin(new_users)]

new_users, len(new_users), len(all_users)

([1409,
  1413,
  1669,
  2438,
  1032,
  650,
  1035,
  1675,
  16,
  272,
  1043,
  1302,
  1565,
  1192,
  683,
  1583,
  1713,
  954,
  2364,
  578,
  1987,
  837,
  2373,
  2120,
  75,
  972,
  1227,
  2256,
  1504,
  997,
  2409,
  506,
  248,
  634],
 34,
 2055)

In [ ]:
#implicit
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['als'], row['actual']), axis=1).mean()

In [ ]:
result_lvl_1['own_recommendations'] = \
            result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['own_recommendations'], row['actual']), axis=1).mean()

In [ ]:
result_lvl_1['similar_items_recommendation'] = \
            result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_recommendation'], row['actual']), axis=1).mean()

In [ ]:
result_lvl_1['similar_items_recommendation'] = \
            result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=200))
result_lvl_1.apply(lambda row: recall_at_k(row['similar_items_recommendation'], row['actual']), axis=1).mean()

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
def new_user_features(data, user_features):
    new_user_features = user_features.merge(data, on='user_id', how='left')
    #Чек
    basket = new_user_features.groupby(['user_id'])['sales_value'].sum().reset_index()
    baskets_qnt = new_user_features.groupby('user_id')['basket_id'].count().reset_index()
    baskets_qnt.rename(columns={'basket_id': 'baskets_qnt'}, inplace=True)
    #Средний недельный чек
    average_basket = basket.merge(baskets_qnt)
    average_basket['average_basket'] = average_basket.sales_value / average_basket.baskets_qnt
    average_basket['sum_per_week'] = average_basket.sales_value / new_user_features.week_no.nunique()
    average_basket = average_basket.drop(['sales_value', 'baskets_qnt'], axis=1)
    user_features = user_features.merge(average_basket)

    return user_features


def new_item_features(data, item_features):
    new_item_features = item_features.merge(data, on='item_id', how='left')
    
    # Цена
    price = new_item_features.groupby('item_id')['sales_value'].sum() \
                            / new_item_features.groupby('item_id')['quantity'].sum()
    price = price.groupby('item_id').mean().reset_index()
    price.columns = ['item_id', 'price']
    price['price'].fillna(0, inplace= True)
    
    #Количество продаж и среднее количество продаж товара

    item_qnt = new_item_features.groupby(['item_id'])['quantity'].count().reset_index()
    item_qnt.rename(columns={'quantity': 'quantity_of_sales'}, inplace=True)
    item_qnt['quantity_of_sales_per_week'] = \
                item_qnt['quantity_of_sales'] / new_item_features['week_no'].nunique()
    item_features = item_features.merge(item_qnt, on='item_id')
    
    return item_features

def train_test_preprocessing(data):    
    users_lvl_2 = pd.DataFrame(data['user_id'].unique())
    users_lvl_2.columns = ['user_id']

    train_users = data_train_lvl_1['user_id'].unique()
    train_users.shape

    users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]
    users_lvl_2_ = users_lvl_2.copy()
    users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
    
    s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'

    users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
    
    users_lvl_2['flag'] = 1

    targets_lvl_2 = data[['user_id', 'item_id']].copy()
    targets_lvl_2.head(2)

    targets_lvl_2['target'] = 1  

    targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

    targets_lvl_2['target'].fillna(0, inplace= True)
    targets_lvl_2.drop('flag', axis=1, inplace=True)

    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

    X = targets_lvl_2.drop('target', axis=1)
    y = targets_lvl_2[['target']]
    
    return X, y


def get_final_recomendation(X_test, test_preds_proba, data_val_lvl_2):
    X_test['predict_proba'] = test_preds_proba

    X_test.sort_values(['user_id', 'predict_proba'], ascending=False, inplace=True)

    result = X_test.groupby('user_id').head(5)

    recs = result.groupby('user_id')['item_id']
    recomendations = []
    for user, preds in recs:
        recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

    recomendations = pd.DataFrame(recomendations)

    result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result_lvl_2.columns=['user_id', 'actual']

    result_lvl_2 = result_lvl_2.merge(recomendations)
    
    return result_lvl_2

In [ ]:
item_features = new_item_features(data_train_lvl_2, item_features)
item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,quantity_of_sales,quantity_of_sales_per_week
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,0,0.0
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,0,0.0
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,0,0.0
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,0,0.0
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,0,0.0
...,...,...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,,0,0.0
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,,0,0.0
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,,0,0.0
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,,0,0.0


In [ ]:
user_features = new_user_features(data_train_lvl_2, user_features)
user_features

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,average_basket,sum_per_week
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.324432,34.091667
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.684286,37.580000
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,2.438000,52.823333
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,7.848624,142.583333
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,2.421525,23.811667
...,...,...,...,...,...,...,...,...,...,...
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494,NaN,0.000000
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496,2.785741,75.215000
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497,3.082063,82.188333
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498,3.456364,6.336667


In [ ]:
X_train, y_train = train_test_preprocessing(data_train_lvl_2)

In [ ]:
cat_feats = X_train.columns[2:].tolist()

In [ ]:
X_train[cat_feats] = X_train[cat_feats].astype('category')
X_test, y_test = train_test_preprocessing(data_val_lvl_2)
X_test[cat_feats] = X_test[cat_feats].astype('category')

In [ ]:
lgb = LGBMClassifier(objective='binary', 
                     #max_depth=7, 
                     categorical_column=cat_feats)
lgb.fit(X_train, y_train)

test_preds_proba = lgb.predict_proba(X_test)[:, 1]

In [ ]:
test_preds_proba

In [ ]:
result_lvl_2 = get_final_recomendation(X_test, test_preds_proba, data_val_lvl_2)

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['recomendations'], row['actual']), axis=1).mean()

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 